In [1]:
import numpy as np
import pickle
import h5py
from temporaldata import Data, RegularTimeSeries, Interval
import glob

In [2]:
def extract_neural_data(data_dict, trials):
    total_df = np.concatenate([df[:-45,:] for df in data_dict['df']])
    total_df[np.isnan(total_df)] = 0
    calcium = RegularTimeSeries(
        df=total_df,
        cue_onsets=np.concatenate([co[:-45,:] for co in data_dict['cue_onsets']]),
        sampling_rate=30,
        domain=trials,
    )
    return calcium

In [3]:
def extract_trials(data_dict):
    boundaries = np.cumsum([0] + [len(trial_df)-45 for trial_df in data_dict['df']]) / 30
    trials = Interval(
        start=boundaries[:-1],
        end=boundaries[1:],
        total_cues=np.squeeze(np.stack(data_dict['total_cues'])),
        rewarded=np.squeeze(data_dict['rewarded']),
        trial_mask=np.squeeze(data_dict['trial_mask']),
    )
    return trials

In [4]:
def extract_behavior(data_dict, trials):
    behavior = RegularTimeSeries(
        position=np.concatenate(data_dict['position']),
        view_angle=np.concatenate(data_dict['view_angle']),
        velocity=np.concatenate(data_dict['velocity']),
        angular_velocity=np.concatenate(data_dict['angular_velocity']),
        sampling_rate=30,
        domain=trials,
    )
    return behavior

In [5]:
files = glob.glob('/home/hice1/smahato7/scratch/dop/data/' + '*.pkl')
for file in files:
    with open(file, 'rb') as f:
        data_dict = pickle.load(f)
        trials = extract_trials(data_dict)
        neural = extract_neural_data(data_dict, trials)
        behavior = extract_behavior(data_dict, trials)
        data = Data(
            session_id=file.split('/')[-1].split('.')[0],
            neural=neural,
            behavior=behavior,
            trials=trials,
            domain=trials,
        )
        with h5py.File('/home/hice1/smahato7/scratch/dop/data/' + data.session_id + '.h5', 'w') as h:
            data.to_hdf5(h)
            print('Saved ' + data.session_id + '.h5')

Saved 651_11-7-16.h5
Saved 946_9-18-17.h5
Saved 556_8-9-16.h5
Saved 947_10-10-17_take_2_new_position.h5
Saved 14974_3-8-16.h5
Saved 150_1-16-18.h5
Saved 1003_1-7-18.h5
Saved 947_8-31-17.h5
Saved 145_1-23-18.h5
Saved 697_12-7-16.h5
Saved 803_2-22-17.h5
Saved 5330_8-1-16_take_2.h5
Saved 14985_6-15-16.h5
Saved 151_1-18-18.h5
Saved 927_8-2-17.h5
Saved 801_3-30-17.h5
Saved 150_1-22-18.h5
Saved 161_4-7-17.h5
Saved 802_5-31-17.h5
Saved 14972_3-10-16.h5
Saved 151_1-17-18.h5
Saved 147_1-19-18.h5
Saved 131_1-8-18.h5
